In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy import stats

In [11]:
# https://stackoverflow.com/questions/21546739/load-data-from-txt-with-pandas
#https://stackoverflow.com/questions/40011531/in-pandas-when-using-read-csv-how-to-assign-a-nan-to-a-value-thats-not-the



# Read data, change missing values to 'NaN'
#Note: All files are unedited from Kaggle

Xtrain_raw = pd.read_csv('Xtrain.txt', sep =" ", index_col=0, dtype='float64',
                    na_values=['?','NaN'])
Xtest_raw = pd.read_csv('Xtest.txt', sep =" ", index_col=0, dtype='float64',
                    na_values=['?','NaN'])

Ytrain_raw = pd.read_csv('Ytrain.txt', sep =",", index_col=0, dtype='float64',
                    na_values=['?','NaN'])

In [12]:
#check the dimensions of the data

Ytrain_raw.shape

(153287, 1)

In [13]:
Xtrain_raw.shape

(153287, 75)

In [14]:
Xtest_raw.shape


(50000, 75)

In [26]:
#https://stackoverflow.com/questions/51207491/function-to-replace-nan-values-in-a-dataframe-with-mean-of-the-related-column
#Replace NaN with column means in Xtrain

Xtrain_filled = Xtrain_raw.apply(lambda x: x.fillna(x.mean())) 
#print(Xtrain_filled.info())

In [27]:
#Do the same for other data-sets

Ytrain_filled = Ytrain_raw.apply(lambda x: x.fillna(x.mean())) 
Xtest_filled = Xtest_raw.apply(lambda x: x.fillna(x.mean())) 

In [28]:
#Drop Junk Data

Xtest_clean=Xtest_filled.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)
Xtrain_clean=Xtrain_filled.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)

In [29]:
#Concatenate the training and 

X_filled = pd.concat([Xtrain_clean, Xtest_clean])
#print('X_filled shape: ', Xtest_filled.shape)


In [30]:
X_sorted = X_filled.sort_index(axis=0)
#X_sorted.info()

In [31]:
#Merge training data sets
train_merged= Xtrain_clean.merge(Ytrain_filled, how='left', on='Id')
print(train_merged)

              #A01      #A02      #B01      #B02      #B03      #B04  \
Id                                                                     
2.0      -0.067006  2.000400 -0.181087 -0.288238 -0.026722 -1.332320   
3.0      -0.238565 -1.755040 -0.562340 -1.943580  0.135810 -0.027425   
4.0      -0.900302  0.760359 -0.566809  0.843982  0.902949 -0.298859   
5.0       0.448940 -0.530155 -0.723218  0.014280  0.175458  0.932137   
7.0      -0.096147  0.123017 -0.269223  0.229115 -0.041637 -0.070465   
...            ...       ...       ...       ...       ...       ...   
177724.0 -0.341901 -0.307778  0.599235  0.580111 -0.033728 -0.038976   
177725.0  1.284220  0.592979 -0.621695 -0.009792 -0.382274 -0.664959   
177727.0  0.454239  1.059110 -0.034778 -0.297135  0.171074 -0.324943   
177729.0 -0.336121  0.218963  0.985771 -0.834321 -1.452110 -0.290119   
177730.0 -0.202946  0.004449 -0.403091  0.019818  1.420010 -1.087600   

              #B05      #B06      #B07      #B08  ...      #F10

In [32]:
#Remove the Id column and reset the index

train_merged_resetindex = train_merged.reset_index()


In [33]:
train_merged_noindex = train_merged_resetindex.drop(['Id'], axis=1)

#print(train_merged_noindex)

In [34]:
#Modeling [some code reused from project 2]
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error

In [35]:
#Split the training sets

y = train_merged['Value'].copy()
X = train_merged.drop(['Value'], axis=1).copy()

In [36]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=32)

In [37]:
# Linear Regression: Fit the model to the training set and get hte RMSE of the predictions 

LinModel = LinearRegression().fit(x_train, y_train)
Lin_predictions = LinModel.predict(x_test)
mean_squared_error(y_test, Lin_predictions, squared=False)

9.641221097353835

In [38]:
#Generate predictions on the test set

Xtest_Lin_predictions = LinModel.predict(Xtest_clean)


In [39]:
#Load the sample predictions
pred = pd.read_csv('pred.txt', sep =",", index_col=0, dtype= {'Value': np.float64})        

In [40]:
sassa_pred_1 = pred.copy()
sassa_pred_1['prediction'] = Xtest_Lin_predictions

In [41]:
sassa_pred_1.to_csv('/Users/Jamie/sassa_pred_1.csv', index = True)